In [32]:
import sys,os
sys.path.append('../OneClickTraining/')
sys.path.append('../Others/')
from all_model_py import *
from others_py import *
import pickle
import pandas as pd
            
            
class ClassifierBase:
    def load_model(self, **model_path):
        pass
            
            
####### classifier 0
####### InitClassifier

class InitClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to initialize conversation'
        self.label_explain = {0: 'next'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        result_dict = {'label':0,'ptp_time':None}
        return result_dict
    

    
    
class StopClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to handle stop logic'
        self.label_explain = {0: 'stop'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        
        result_dict = {'label':0,'ptp_time':None}
        return result_dict
    
    
    






class Node:
    def __init__(self, node_name, msg_path=None):
        self.name = node_name
        self._load_message(msg_path)
        self.canJump = False
        self.sentiment = 1
        #print('{} is initialized'.format(self.name))

        
        
    def summary(self):
        return {'node_name': self.name, 
                'description':self.describe, 
                'class_name':self.__class__.__name__, 
                'model': self.model_name}
    
    def _triger_jump(self):
        if self.canJump is True:
            # jump trigger
            if self.output_label == 1 and self.sentiment >=2: 
                self.output_label = 1001
        else:
            return None
    
    
    
    def process(self, sentence, model_dict):
        model = model_dict[self.model_name]
        clf = model.classify(sentence)
        
        self.output_label = clf['label']
        # jump trigger
        self._triger_jump()
        self.ptp_time = clf.get('ptp_time')
        return self.output_label, self.ptp_time
    
    
    def _load_message(self, msg_path):
        self.messages = pd.read_csv(msg_path, encoding='utf8')
        self.messages = self.messages[self.messages['node_name'] == self.name]
        self.messages.label = self.messages.label.astype('int')
        self.messages.sentiment = self.messages.sentiment.astype('int')
        
        
    def get_response(self, label):
        """
        return response by label
        """
        
        df = self.messages[self.messages.label == label]
        max_sentiment = np.max(df.sentiment.values)
        
        if self.sentiment > max_sentiment:
            sentiment = max_sentiment
        else:
            sentiment = self.sentiment
            
        df = df[df.sentiment == sentiment]
        # enable random extract
        try:
            df = df.sample(frac=1)
        except ValueError:
            response = 'current node name is{}，ouput label is{},sentiment is{}, no message has been set'.format(self.name,label,sentiment)
            print(response)
            return response
        try:
            response = df.message.values[0]
            add_sentiment = df.add_sentiment.values[0]
        except IndexError:
            response = 'current node name is{}，ouput label is{},sentiment is{}, no message has been set'.format(self.name,label,sentiment)
            print(response)
            return response
        self.sentiment += add_sentiment
        return response


        
        

        
###################### Node 0  #########################

    
class S1_N0(Node):
    def __init__(self, msg_path):
        super().__init__('s0', msg_path)
        self.describe = 'Init node'
        self.model_name = 'InitClassifier'

        
###################### Node 1  #########################
class S1_N1(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n1_identity_q', msg_path)
        self.describe = 'Verify Identify'
        self.model_name = 'IDClassifier'
        
        
###################### Node 1  #########################
class S1_N2(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n2_confirmLoan_q', msg_path)
        self.describe = 'Verify Identify'
        self.model_name = 'ConfirmLoan'
        self.canJump = True
                

                
#######################  Node 2  #############################        
class S1_N5(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n5_ifAcquainted_q', msg_path)
        self.describe = 'Ask if know debtor'
        self.model_name = 'IfKnowDebtor'
        
        
##########################  Node 3  ##########################        
class S1_N15(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n15_verifyWill_q', msg_path)
        self.describe = 'Verify willing to pay'
        self.model_name = 'WillingToPay'
        self.canJump = True
                
       

        
#########################  Node 7  ###########################        
class S1_N25(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n25_cutDebt_q', msg_path)
        self.describe = 'ask if accept less amount'
        self.model_name = 'CutDebt'
        self.canJump = True
        
        
        
                
        
#########################  Node 8  ###########################        
class S1_N32(Node):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n32_splitDebt_q', msg_path)
        self.describe = 'ask if accept installment'
        self.model_name = 'Installment'
        self.canJump = True
        
        
        
############################## STOP NODE ########################
class NodeStop(Node):
    def __init__(self, node_name, msg_path):
        super().__init__(node_name, msg_path)
        self.model_name = 'StopClassifier'



class S1_N101(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n101_ifAcquainted_s', msg_path)
        self.describe = 'inform phone recipient'

        
        
class S1_N102(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n102_ifAcquainted_s', msg_path)
        self.describe = 'do not know debtor'
    
 
        
class S1_N103(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n103_paymentChannel_s', msg_path)
        self.describe = 'notify methods of paying'
    

        
class S1_N104(NodeStop):
    def __init__(self, msg_path):
        super().__init__('cf_s1_n104_paymentChannel_s', msg_path)
        self.describe = 'notify methods of paying'
    
        
class S1_N105(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n105_noResult_s',msg_path)
        self.describe = 'no result'
        
    

class S1_N106(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n106_paymentChannel_s',msg_path)
        self.describe = 'notify methods of paying'
        
        
        
class S1_N108(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n108_noResult_s',msg_path)
        self.describe = 'no result'
        
class S1_N109(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n109_scheduleCall_s',msg_path)
        self.describe = 'no result'
        
class S1_N110(NodeStop):
    def __init__(self,msg_path):
        super().__init__('cf_s1_n110_recordWechat_s',msg_path)
        self.describe = 'no result'
        
    

        
    
########################################################################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
      
        
class TreeBase:
    def __init__(self, start_node='s0', ):
        self.current_node_name = start_node
        self.fc_path = []
        self.all_path = []
        
        
    
class TreeStage1(TreeBase):
    def __init__(self, start_node='s0',graph_path='',msg_path='',debug=False):
        super().__init__(start_node=start_node)
        self._build_node(msg_path)
        self._build_graph(graph_path)
        self.debug = debug
        
    def _build_node(self,msg_path):
        self.messages = pd.read_csv(msg_path,encoding='utf8')
        self.nodes = {
        's0':S1_N0(msg_path),
        'cf_s1_n1_identity_q':S1_N1(msg_path),
        'cf_s1_n15_verifyWill_q':S1_N15(msg_path),
        'cf_s1_n101_ifAcquainted_s':S1_N101(msg_path),
        'cf_s1_n102_ifAcquainted_s':S1_N102(msg_path),
        'cf_s1_n103_paymentChannel_s':S1_N103(msg_path),
        'cf_s1_n104_paymentChannel_s':S1_N104(msg_path),
        'cf_s1_n105_noResult_s':S1_N105(msg_path),
        'cf_s1_n106_paymentChannel_s':S1_N106(msg_path),
        'cf_s1_n108_noResult_s':S1_N108(msg_path),
        'cf_s1_n109_scheduleCall_s':S1_N109(msg_path),
        'cf_s1_n110_recordWechat_s':S1_N110(msg_path),
        'cf_s1_n2_confirmLoan_q': S1_N2(msg_path),
        'cf_s1_n25_cutDebt_q':S1_N25(msg_path),
        'cf_s1_n32_splitDebt_q':S1_N32(msg_path),
        'cf_s1_n5_ifAcquainted_q':S1_N5(msg_path),} 
        
    def _build_graph(self,graph_path):
        self.df_mapping = pd.read_csv(graph_path)
        gp = self.df_mapping.groupby('node_name')
        self.mapping = {}
        for each in gp:
            df_tmp = each[1]
            df_tmp = df_tmp.set_index('label')
            self.mapping.update({each[0]:df_tmp.T.to_dict()})
        
        

        
    def _updates(self, _label):
        """
        update fc_path, all_path, current_node_name
        return current node, response
        """
        cur_node = self.nodes[self.current_node_name]
        
        # get current response
        response = cur_node.get_response(_label)

        
        # get next node_name
        if self.mapping.get(self.current_node_name) is not None:
            next_node_name = self.mapping.get(self.current_node_name)[_label]['connection']
        else:
            next_node_name = None
            
        if self.debug:
            response = response + '<-current node is: {}->'.format(self.current_node_name)
            response = response + '<-output label is: {}->'.format(_label)
            response = response + '<-next node is: {}->'.format(next_node_name)
        return response, next_node_name
        
        
        
    def process(self, sentence, model_dict):
        current_node = self.nodes[self.current_node_name] 
        if current_node.model_name == 'StopClassifier':
            return 'end'
        _label,_ptp = current_node.process(sentence, model_dict)

        response,next_node_name = self._updates(_label)
        
        if next_node_name is None:
            return 'end'
        else:
            self.current_node_name = next_node_name
        return response
    
    def ttest(self, sentence, model_dict,label):
        current_node = self.nodes[self.current_node_name] 
        if current_node.model_name == 'StopClassifier':
            return 'end'
        _label,_ptp = current_node.process(sentence, model_dict)
        _label = label

        response,next_node_name = self._updates(_label)
        
        if next_node_name is None:
            return 'end'
        else:
            self.current_node_name = next_node_name
        return response

In [33]:
models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../savedModel/{}/{}.pickle'
model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一遍')
model_dict['InitClassifier'] = InitClassifier()
model_dict['StopClassifier'] = StopClassifier()

In [37]:
t1 = TreeStage1(graph_path='../../data/TreeModel/treeConnection.csv',msg_path='../../data/TreeModel/node_message.csv',debug=True)

In [41]:
print(t1.current_node_name)
print(t1.process('去吃粑粑',model_dict))
print(t1.current_node_name)

cf_s1_n1_identity_q
你说这些乱七八糟的都没有用的，这个该还的你肯定是跑不掉的。你现在告诉我你是罗巍先生吗？<-current node is: cf_s1_n1_identity_q-><-output label is: 109-><-next node is: cf_s1_n1_identity_q->
cf_s1_n1_identity_q


In [12]:
t1.messages

,node_name,label,sentiment,message
0,s0,0,1,你好，这里是江苏逸能催收公司，请问是罗巍先生吗？
1,cf_s1_n1_identity_q,1,1,不好意思，打扰了，请问您认识罗先生吗！
2,cf_s1_n1_identity_q,0,1,您好，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500...
3,cf_s1_n1_identity_q,101,1,是这样的，您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息15...
4,cf_s1_n1_identity_q,103,1,我们这里是江苏逸能催收公司的，请问您是罗巍先生吗？
5,cf_s1_n1_identity_q,104,1,那好吧，那我过一会儿再给您打过去。再见。
6,cf_s1_n1_identity_q,107,1,我这边是江苏逸能催收公司的催收员。您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5...
7,cf_s1_n1_identity_q,109,1,请你不要说这些有的没的，您是罗巍先生吗？
8,cf_s1_n1_identity_q,110,1,你说这些乱七八糟的都没有用的，这个该还的你肯定是跑不掉的。你现在告诉我你是罗巍先生吗？
9,cf_s1_n2_confirmLoan_q,0,1,您之前借贷了5万块钱，截至今天已经逾期47天，逾期总欠款5万2千块钱，其中利息1500，滞纳...


In [37]:
t1.nodes['cf_s1_n2_confirmLoan_q'].get_response(2)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
print(t1.process('是你大爷',model_dict))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
b = t1.nodes['s0'].messages
b

,node_name,label,sentiment,message
0,s0,0,1,你好，这里是江苏逸能催收公司，请问是罗巍先生吗？


In [ ]:
def get_response(self, label, sentiment=1):
        """
        return response by label
        """
        df = self.messages[(self.messages.label == label) & (self.messages.sentiment == sentiment)]
        response = df.message.values[0]
        self.entry_counter += 1
        return response

In [30]:
model_dict['IfKnowDebtor'].classify('可以')

{'label': 104, 'pred_prob': array([[0.39139882, 0.10326868, 0.5053325 ],
        [0.43655833, 0.18918957, 0.3742521 ],
        [0.5465552 , 0.21296955, 0.24047525]]), 'av_pred': array([0.45817078, 0.16847594, 0.37335328])}

In [34]:
class p1:
    def __init__(self, a):
        self.a =a
class p2(p1):
    pass


In [35]:
p = p2(a=2)

In [36]:
p.a

2

In [13]:
a = []

In [14]:
a[0]

IndexError: list index out of range